Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [1]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy


## The *Nim* and *Nimply* classes

In [2]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [3]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [4]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [5]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [6]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [7]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


In [8]:
import random


def generate_individual():
    # Genera una strategia casuale
    lambda_value = random.uniform(0.0, 1.0)  # Genera un valore di λ casuale tra 0 e 1

    individual = {"lambda": lambda_value}

    return individual


def generate_population(population_size):
    # Genera una popolazione di individui
    population = [generate_individual() for _ in range(population_size)]
    return population

def mossa(state: Nim,individual) :
    analysis = analize(state)
        
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    max_ply = max(spicy_moves, key=lambda ply: (1 - individual["lambda"]) * analysis["possible_moves"][ply])
    return max_ply
               

def fitness (individual):
    strategy = (mossa, pure_random)
    num_partite_vinte=0
    for _ in range(0,5):
        nim = Nim(5)
        player = 0
        while nim:
            if strategy[player] == mossa :
                ply=strategy[player](nim,individual)
                nim.nimming(ply)
            else:
                ply = strategy[player](nim)
                nim.nimming(ply)
            player = 1 - player
        if player == 0:
            num_partite_vinte+=1
        else:
            if num_partite_vinte == 0:
                num_partite_vinte-=1
    return num_partite_vinte

def fitness_population(population) :
    max_fitness=0
    temp_population = []
    for individual in population:
        num_partite_vinte=fitness(individual)
        temp_population.append((individual,num_partite_vinte))
        if max_fitness < num_partite_vinte :
            max_fitness = num_partite_vinte
    final_population = [ind for ind,num_partite_vinte in temp_population if num_partite_vinte == max_fitness]
    return final_population


def crossover(parent1,parent2):
    binary_representation_1 = bin(int(parent1["lambda"] * 1000))[2:]
    binary_representation_2 = bin(int(parent2["lambda"]*1000))[2:]
    casual_point = random.randint(1,min(len(binary_representation_1),len(binary_representation_2)))
    figlio_lambda = binary_representation_1[:casual_point] + binary_representation_2[casual_point:]
    figlio_lambda_decimal = int(figlio_lambda,2) / 1000
    figlio = {"lambda":figlio_lambda_decimal}
    return figlio


def es(num_generations) :
    population = generate_population(100)
    for _ in range(num_generations):
        final_population = fitness_population(population)
        new_population = []
        while len(new_population) < len(population):
            parent1 = random.choice(final_population)
            parent2 = random.choice(final_population)
            figlio = crossover(parent1,parent2)
            new_population.append(figlio)
    population=new_population
    return population

## Oversimplified match

In [9]:
# logging.getLogger().setLevel(logging.INFO)

# strategy = (optimal, pure_random)

# nim = Nim(5)
# logging.info(f"init : {nim}")
# player = 0
# while nim:
#     ply = strategy[player](nim)
#     logging.info(f"ply: player {player} plays {ply}")
#     nim.nimming(ply)
#     logging.info(f"status: {nim}")
#     player = 1 - player
# logging.info(f"status: Player {player} won!")

#valutazione evolutionary algorithm
evolutionary_population = es(5)
logging.getLogger().setLevel(logging.INFO)

strategy = (mossa, gabriele)
individual=random.choice(evolutionary_population)
nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    if strategy[player] == mossa :
        ply=strategy[player](nim,individual)
        nim.nimming(ply)
    else:
        ply = strategy[player](nim)
        nim.nimming(ply)
    logging.info(f"ply: player {player} plays {ply}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=2)
INFO:root:ply: player 1 plays Nimply(row=0, num_objects=1)
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=1)
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=2)
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=2)
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=1)
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=6)
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=1)
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=8)
INFO:root:status: Player 0 won!
